In [29]:
import pymysql, os
import numpy as np
from astropy import units as u
from astropy.coordinates import SkyCoord
import pandas, alminer, glob
import matplotlib.pyplot as plt

In [13]:
from astropy.cosmology import FlatLambdaCDM
cosmo = FlatLambdaCDM(H0=70, Om0=0.3)

In [1]:
from skaha.session import Session
import nest_asyncio
nest_asyncio.apply()

# Read in sample to be reduced

In [90]:
fpath = '/arc/projects/salvage/ALMA_reduction/samples/'
file  =  'pilot_sample.txt'

objID_sample, year_sample, name_sample, muid_sample, guid_sample, auid_sample, proj_sample = np.loadtxt(fpath+file, unpack = True, dtype = str, usecols = [0,11,12,13,14,15,16])
z_sample, mass_sample, rpetro_sample, ra_sample, dec_sample, res_sample, mrs_sample, AL_sample, AC_sample, TP_sample = np.loadtxt(fpath+file, unpack = True, dtype = float, usecols = [1,2,3,4,5,6,7,8,9,10])

In [108]:
dec_sample[objID_sample=='587727944034091201']

array([2.30472222])

In [10]:
objID_sample

array(['587731173842027508', '587731173842027508', '587731185665638418',
       '587726033309335775', '587726032772333771', '587727944033960150',
       '587731511545626673', '587731187797393472', '587731511542022280',
       '588010359075700997', '588010359075700997', '588010359075700997',
       '588015508731723912', '587731512619696292', '587741490891325518',
       '587726033335550094', '587726033335550094', '587727221402042402',
       '587727221396013696', '587727944034091201', '587727944034091092',
       '587727944034091239', '588015510364880957', '587741421099155681',
       '587742060530696249', '587731186187632690'], dtype='<U28')

In [17]:
objID, M_H2 = np.loadtxt('/arc/projects/salvage/ALMA_reduction/gas_mass.txt', unpack = True, usecols = [0,1], dtype = str)

In [18]:
for i in range(len(objID)):
    print(objID[i], M_H2[i])

587731173842027508 10.032787859826723
587731173842027508 10.032787859826723
587731185665638418 10.452379874746919
587726033309335775 -inf
587731511545626673 9.324301909608131
587731187797393472 9.345843206175083
587731511542022280 8.904965482454399
588015508731723912 8.97804738047872
587731512619696292 9.368323484768842
587741490891325518 10.100829629437847
587726033335550094 10.64403222556311
587726033335550094 10.64403222556311
587727221402042402 9.273333573492518
587727221396013696 9.344404424956812
588015510364880957 8.99166954393916
587741421099155681 8.94696891186483
587742060530696249 9.295907516012456
587731186187632690 9.564979412680053


# ALMiner Download and Unzip with headless jobs

In [19]:
# select appropriate resources

image = "images.canfar.net/skaha/astroml:latest"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh'
ram=8
cores=2

session_ids = []

rerun_targets = ['587726032772333771','587727944033960150','587727944034091201','587727944034091092']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    RA = ra_sample[i]
    DEC = dec_sample[i]
    NAME = name_sample[i]
    MUID = muid_sample[i]
    GUID = guid_sample[i]
    AUID = auid_sample[i]
    PROJ = proj_sample[i]

    # running only on problem galaxies
    if ID not in rerun_targets:
        continue

    TAR1 = PROJ + '_uid___' + MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4] + '_001_of_001.tar'
    TAR2 = PROJ + '_uid___' + MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4] + '_auxiliary.tar'
    ASDM = PROJ + '_uid___' + AUID.split('/')[2] + '_' + AUID.split('/')[3] + '_' + AUID.split('/')[4] + '.asdm.sdm.tar'
    #ASDM = PROJ + '_uid___' + AUID.split('/')[2] + '_' + AUID.split('/')[3] + '_*.asdm.sdm.tar'

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{ID} {NAME} {TAR1} {TAR2} {ASDM}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-13 22:35:50,527 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-13 22:35:50,527 INFO Creating 1 session(s) with parameters:
2024-03-13 22:35:50,530 - skaha-client-skaha.session - INFO - {'name': '587726032772333771', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587726032772333771 COSMOS-3666 2018.1.01128.S_uid___A001_X134b_X1a2_001_of_001.tar 2018.1.01128.S_uid___A001_X134b_X1a2_auxiliary.tar 2018.1.01128.S_uid___A002_Xd69abb_X2ca4.asdm.sdm.tar', 'env': {}}
2024-03-13 22:35:50,530 INFO {'name': '587726032772333771', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587726032772333771 COSMOS-3666 2018.1.01128.S_uid___A001_X134b_X1a2_001_of_001.tar 2018.1.01128.S_uid___A001_X

Sesion ID: c316k2wb


2024-03-13 22:35:54,738 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-13 22:35:54,738 INFO Creating 1 session(s) with parameters:
2024-03-13 22:35:54,741 - skaha-client-skaha.session - INFO - {'name': '587727944034091201', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587727944034091201 COSMOS-1197519 2018.1.01852.S_uid___A001_X133d_X4226_001_of_001.tar 2018.1.01852.S_uid___A001_X133d_X4226_auxiliary.tar 2018.1.01852.S_uid___A002_Xd9668b_Xaccf.asdm.sdm.tar', 'env': {}}
2024-03-13 22:35:54,741 INFO {'name': '587727944034091201', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587727944034091201 COSMOS-1197519 2018.1.01852.S_uid___A001_X133d_X4226_001_of_001.tar 2018.1.01852.S_uid

Sesion ID: h14b5ylr


2024-03-13 22:35:56,414 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-13 22:35:56,414 INFO Creating 1 session(s) with parameters:
2024-03-13 22:35:56,416 - skaha-client-skaha.session - INFO - {'name': '587727944034091092', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587727944034091092 COSMOS-0969208 2018.1.01852.S_uid___A001_X133d_X4226_001_of_001.tar 2018.1.01852.S_uid___A001_X133d_X4226_auxiliary.tar 2018.1.01852.S_uid___A002_Xd9668b_Xaccf.asdm.sdm.tar', 'env': {}}
2024-03-13 22:35:56,416 INFO {'name': '587727944034091092', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/download_and_unzip.sh', 'args': '587727944034091092 COSMOS-0969208 2018.1.01852.S_uid___A001_X133d_X4226_001_of_001.tar 2018.1.01852.S_uid

Sesion ID: pmb0uea0
Sesion ID: mxdwer97


In [26]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

2024-03-14 01:14:12,553 - skaha-client-skaha.session - ERROR - 404 Client Error: Not Found for url: https://ws-uv.canfar.net/skaha/v0/session/c316k2wb?view=event
2024-03-14 01:14:12,553 ERROR 404 Client Error: Not Found for url: https://ws-uv.canfar.net/skaha/v0/session/c316k2wb?view=event


IndexError: list index out of range

In [22]:
for sid in session_ids:

    print(session.logs(sid)[sid])

## Run ScriptForPI.py

In [30]:
# select appropriate resources
image = "images.canfar.net/casa-5/casa:5.6.1-8-pipeline"
#image = "images.canfar.net/casa-4/casa:4.2.2-pipe"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh'
ram=8
cores=2

session_ids = []

rerun_targets = ['587726032772333771','587727944033960150','587727944034091201','587727944034091092']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # running only on problem galaxies
    if ID not in rerun_targets:
        continue

    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{ID} {PROJ} {PATH}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-14 01:16:01,626 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-14 01:16:01,626 INFO Creating 1 session(s) with parameters:
2024-03-14 01:16:01,627 - skaha-client-skaha.session - INFO - {'name': '587726032772333771', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587726032772333771 2018.1.01128.S /arc/projects/salvage/ALMA_data/587726032772333771/2018.1.01128.S/science_goal.uid___A001_X134b_X1a0/group.uid___A001_X134b_X1a1/member.uid___A001_X134b_X1a2', 'env': {}}
2024-03-14 01:16:01,627 INFO {'name': '587726032772333771', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587726032772333771 2018.1.01128.S /arc/projects/salvage/ALMA_data/587726032772333771/20

Sesion ID: d8w39sjp


2024-03-14 01:16:07,633 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-14 01:16:07,633 INFO Creating 1 session(s) with parameters:
2024-03-14 01:16:07,635 - skaha-client-skaha.session - INFO - {'name': '587727944033960150', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587727944033960150 2018.1.01128.S /arc/projects/salvage/ALMA_data/587727944033960150/2018.1.01128.S/science_goal.uid___A001_X134b_X1ac/group.uid___A001_X134b_X1ad/member.uid___A001_X134b_X1ae', 'env': {}}
2024-03-14 01:16:07,635 INFO {'name': '587727944033960150', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587727944033960150 2018.1.01128.S /arc/projects/salvage/ALMA_data/587727944033960150/20

Sesion ID: ogf0ta3m


2024-03-14 01:16:10,352 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-14 01:16:10,352 INFO Creating 1 session(s) with parameters:
2024-03-14 01:16:10,354 - skaha-client-skaha.session - INFO - {'name': '587727944034091201', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587727944034091201 2018.1.01852.S /arc/projects/salvage/ALMA_data/587727944034091201/2018.1.01852.S/science_goal.uid___A001_X133d_X4224/group.uid___A001_X133d_X4225/member.uid___A001_X133d_X4226', 'env': {}}
2024-03-14 01:16:10,354 INFO {'name': '587727944034091201', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587727944034091201 2018.1.01852.S /arc/projects/salvage/ALMA_data/587727944034091201

Sesion ID: sv8l0vbz


2024-03-14 01:16:12,842 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-14 01:16:12,842 INFO Creating 1 session(s) with parameters:
2024-03-14 01:16:12,843 - skaha-client-skaha.session - INFO - {'name': '587727944034091092', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587727944034091092 2018.1.01852.S /arc/projects/salvage/ALMA_data/587727944034091092/2018.1.01852.S/science_goal.uid___A001_X133d_X4224/group.uid___A001_X133d_X4225/member.uid___A001_X133d_X4226', 'env': {}}
2024-03-14 01:16:12,843 INFO {'name': '587727944034091092', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/restore_calibration.sh', 'args': '587727944034091092 2018.1.01852.S /arc/projects/salvage/ALMA_data/587727944034091092

Sesion ID: micvdx3h


In [314]:
print(session.info(session_id))

[{'id': 'bc0qf1d5', 'userid': 'swilkinson', 'appid': '<none>', 'image': 'images.canfar.net/casa-5/casa:5.6.1-8-pipeline', 'type': 'headless', 'status': 'Running', 'name': '587726032772333771-1', 'startTime': '2024-02-13T08:35:50Z', 'expiryTime': '2024-02-27T08:35:50Z', 'connectURL': 'not-applicable', 'requestedRAM': '16G', 'requestedCPUCores': '2', 'requestedGPUCores': '0', 'ramInUse': '636M', 'gpuRAMInUse': '<none>', 'cpuCoresInUse': '0.687', 'gpuUtilization': '<none>'}]


In [32]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

goc7ljwv Pending
ec5e1n4v Pending
kvfq25bw Pending
cko88rg1 Pending
zbipmklb Pending
y2z91qcw Pending
mhg00n32 Pending
gm19yg0s Pending
s0j78dh8 Pending
twcu38ko Pending
al6jr9qg Pending
x8bixstp Pending
f6f5kn7g Pending
yej4st1o Pending
mrnoavyh Pending
j4byp8es Pending
p9vvqczr Pending
irsvfd0i Pending
ndj7agnm Pending
ifa1u64c Pending
y330i3i2 Pending
qzr7njke Pending
yvrqguv0 Pending
aiqezht4 Pending


In [388]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

## Make Measurement Set Key

In [126]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

ms_key_out = 'ms_file_key_salvage.txt'
ms_key_tmp = 'ms_file_key_template.txt'

# prep by copy current ms key to a template file with "__DATA__" where data should go (DONE)

# get template table (formatting with no file paths)
out = open(key_dir + ms_key_tmp, 'r')
out_data = out.read()
out.close()

# wipe old content from output file
out = open(key_dir + ms_key_out, 'w')
out.close()

# initialize objID input tracker
objID_in_file = []

# initialize string
out_str = ''

for i in range(len(objID_sample)):

    ID = objID_sample[i]
    MUID = muid_sample[i]
    PROJ = proj_sample[i]

    UID =  MUID.split('/')[2] + '_' + MUID.split('/')[3] + '_' + MUID.split('/')[4]

    # search all directories in the ALMA project folder for the relevant data
    for root, dirs, files in os.walk(f"/arc/projects/salvage/ALMA_data/{ID}/{PROJ}/"):
        if "member.uid___"+UID in dirs:
            PATH = os.path.join(root, "member.uid___"+UID)

    # prep file path for PHANGS-ALMA pipeline
    ms_root = '/arc/projects/salvage/ALMA_data/'
    ms_filepath = PATH.replace(ms_root, '') # wipe ms_root directory so it can be added separately
    ms_filepath += '/calibrated/' # add calibrated on the end so that it points to the calibrated data

    # until I discover more nuance, take all measurement set files from this directory and let PHANGS deal with them
    ms_files = glob.glob(ms_root + ms_filepath + '*.ms*')

    print(ms_root + ms_filepath, os.path.isdir(ms_root + ms_filepath), len(ms_files))

    if len(ms_files)<1:
        #checking if files are missing because of bug in previous stages or this one...
        print('No MS files found in ', ms_filepath)

    # check if there are visibilities for this target from another project and add on top if there is...
    init = len(np.array(objID_in_file)[np.array(objID_in_file)==ID])

    # add each measurement set to ms key
    for j in range(init, init + len(ms_files)):

        ms_path = ms_files[j-init].replace(ms_root, '') # wipe ms_root directory so it can be added separately
        obs_num = j+1 # record different measurement sets as different observations
    
        out_str += f"{ID} {ID}_{obs_num} all 12m {obs_num} {ms_path}\n" 
        # need to accomodate other observations of the same objID?

        objID_in_file.append(ID)

# replace placeholder "__DATA__" in template with formatted data
out_data = out_data.replace('__DATA__', out_str)
    
# write to new key file
out = open(key_dir + ms_key_out, 'w')
out.write(out_data)
out.close()

/arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00541.S/science_goal.uid___A001_X133d_Xea1/group.uid___A001_X133d_Xea2/member.uid___A001_X133d_Xea3/calibrated/ True 2
/arc/projects/salvage/ALMA_data/587731173842027508/2018.1.00558.S/science_goal.uid___A001_X133d_Xf47/group.uid___A001_X133d_Xf48/member.uid___A001_X133d_Xf49/calibrated/ True 2
/arc/projects/salvage/ALMA_data/587731185665638418/2018.1.00558.S/science_goal.uid___A001_X133d_Xf43/group.uid___A001_X133d_Xf44/member.uid___A001_X133d_Xf45/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587726033309335775/2018.1.01852.S/science_goal.uid___A001_X133d_X4224/group.uid___A001_X133d_X4225/member.uid___A001_X133d_X4226/calibrated/ True 3
/arc/projects/salvage/ALMA_data/587726032772333771/2018.1.01128.S/science_goal.uid___A001_X134b_X1a0/group.uid___A001_X134b_X1a1/member.uid___A001_X134b_X1a2/calibrated/ True 1
/arc/projects/salvage/ALMA_data/587727944033960150/2018.1.01128.S/science_goal.uid___A001_X134b_X1ac/group.uid__

## Make distance key

In [41]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

dist_key_out = 'distance_key_salvage.txt'
dist_key_tmp = 'distance_key_template.txt'

# prep by copy current distance key to a template file with "__DATA__" where data should go (DONE)

# get template table (formatting with no coordinates)
out = open(key_dir + dist_key_tmp, 'r')
out_data = out.read()
out.close()

# initialize string
out_str = ''

for i in range(len(objID_sample)):

    ID   = objID_sample[i]
    NAME = name_sample[i]
    Z    = z_sample[i]

    DIST = cosmo.angular_diameter_distance(Z).value

    out_str += f"{ID},{DIST}\n" 
    # need to accomodate other observations of the same objID?

# replace __DATA__ in template with formatted data
out_data = out_data.replace('__DATA__', out_str)

# write to new key file
out = open(key_dir + dist_key_out, 'w')
out.write(out_data)
out.close()

## Make target definitions key

In [42]:
key_dir = '/arc/projects/salvage/phangs_imaging_scripts/phangs-alma_keys/'

targ_key_out = 'target_definitions_salvage.txt'
targ_key_tmp = 'target_definitions_template.txt'

# prep by copy current target definitions key to a template file with "__DATA__" where data should go (DONE)

# prep for sys vel calculation
restfreq = 115.27120 * u.GHz  # rest frequency of 12 CO 1-0 in GHz
freq_to_vel = u.doppler_radio(restfreq)

# get template table (formatting with no coordinates)
out = open(key_dir + targ_key_tmp, 'r')
out_data = out.read()
out.close()

# initialize string
out_str = ''

# format the ra and dec's for html table
for i in range(len(objID_sample)):

    ID   = objID_sample[i]
    NAME = name_sample[i]
    Z    = z_sample[i]
    RA   = ra_sample[i]
    DEC  = dec_sample[i]

    c = SkyCoord(ra= RA*u.degree, dec= DEC*u.degree, frame='icrs')
    coord_str = c.to_string('hmsdms')

    sys_vel = (restfreq/(1+Z)).to(u.km / u.s, equivalencies=freq_to_vel).value
    velwidth = 1000

    out_str += f"{ID} {coord_str} {sys_vel} {velwidth}\n" 
    # need to accomodate other observations of the same objID?

# replace __DATA__ in template with formatted data
out_data = out_data.replace('__DATA__', out_str)

# write to new key file
out = open(key_dir + targ_key_out, 'w')
out.write(out_data)
out.close()

## Run PHANGS Pipeline

In [127]:
# select appropriate resources

#image = "images.canfar.net/skaha/astroml:latest"
image = "images.canfar.net/casa-6/casa:6.5.6-22"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh'
ram=8
cores=2

session_ids = []

rerun_targets = ['587726032772333771','587727944033960150','587727944034091201','587727944034091092']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    NAME = name_sample[i]

    # running only on problem galaxies
    #if ID not in rerun_targets:
    #    continue
    
    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{NAME} {ID}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-15 22:40:51,941 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-15 22:40:51,941 INFO Creating 1 session(s) with parameters:
2024-03-15 22:40:51,943 - skaha-client-skaha.session - INFO - {'name': '587731173842027508', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587731173842027508_8623-6104 587731173842027508', 'env': {}}
2024-03-15 22:40:51,943 INFO {'name': '587731173842027508', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587731173842027508_8623-6104 587731173842027508', 'env': {}}
2024-03-15 22:40:57,013 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-15 22:40:57,013 INFO Creating 1 session(s) with parameters:
2024-03-15 22:40:

Sesion ID: l3p15o7u


2024-03-15 22:40:59,906 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-15 22:40:59,906 INFO Creating 1 session(s) with parameters:
2024-03-15 22:40:59,907 - skaha-client-skaha.session - INFO - {'name': '587731185665638418', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '8655-3701 587731185665638418', 'env': {}}
2024-03-15 22:40:59,907 INFO {'name': '587731185665638418', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '8655-3701 587731185665638418', 'env': {}}


Sesion ID: homy2yos


2024-03-15 22:41:01,024 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-15 22:41:01,024 INFO Creating 1 session(s) with parameters:
2024-03-15 22:41:01,026 - skaha-client-skaha.session - INFO - {'name': '587726033309335775', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': 'COSMOS-1411106 587726033309335775', 'env': {}}
2024-03-15 22:41:01,026 INFO {'name': '587726033309335775', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': 'COSMOS-1411106 587726033309335775', 'env': {}}


...

Sesion ID: ktttdjba


2024-03-15 22:41:35,503 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-15 22:41:35,503 INFO Creating 1 session(s) with parameters:
2024-03-15 22:41:35,506 - skaha-client-skaha.session - INFO - {'name': '587742060530696249', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587742060530696249_8450-6102 587742060530696249', 'env': {}}
2024-03-15 22:41:35,506 INFO {'name': '587742060530696249', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587742060530696249_8450-6102 587742060530696249', 'env': {}}


Sesion ID: kd6tnkeh


2024-03-15 22:41:36,902 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-15 22:41:36,902 INFO Creating 1 session(s) with parameters:
2024-03-15 22:41:36,904 - skaha-client-skaha.session - INFO - {'name': '587731186187632690', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587731186187632690_8616-9102 587731186187632690', 'env': {}}
2024-03-15 22:41:36,904 INFO {'name': '587731186187632690', 'image': 'images.canfar.net/casa-6/casa:6.5.6-22', 'cores': 2, 'ram': 8, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_pipeline.sh', 'args': '587731186187632690_8616-9102 587731186187632690', 'env': {}}


Sesion ID: pbw3dp3b
Sesion ID: xc1d1rup


In [107]:
# loop over galaxies and launch jobs
for i in range(len(objID_sample[0:10])):

    ID = objID_sample[0:10][i]
    NAME = name_sample[0:10][i]

    print(NAME, ID)

587731173842027508_8623-6104 587731173842027508
8623-6104 587731173842027508
8655-3701 587731185665638418
COSMOS-1411106 587726033309335775
COSMOS-3666 587726032772333771
COSMOS-20062 587727944033960150
587731511545626673_8081-3704 587731511545626673
587731187797393472_8615-3703 587731187797393472
8077-9101 587731511542022280
zwicky_3146 588010359075700997


In [131]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

l3p15o7u Succeeded
homy2yos Succeeded
uqf3ozzb Succeeded
tskcab2s Running
tpcohsfq Succeeded
y0am6u63 Succeeded
y5ec2942 Succeeded
uxx27p8r Succeeded
pq3hw0mj Succeeded
kj85e6j8 Succeeded
iwtiyyi5 Succeeded


2024-03-15 23:42:08,863 - skaha-client-skaha.session - ERROR - 404 Client Error: Not Found for url: https://ws-uv.canfar.net/skaha/v0/session/qr3rdmzx?view=event
2024-03-15 23:42:08,863 ERROR 404 Client Error: Not Found for url: https://ws-uv.canfar.net/skaha/v0/session/qr3rdmzx?view=event


IndexError: list index out of range

In [217]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

Execute bash script.
NAME = 587731173842027508_8623-6104
Running CASA pipeline...
--> CrashReporter initialized.
casalogger-x86_64.AppImage: cannot connect to X server 

optional configuration file config.py not found, continuing CASA startup without it
Using user-supplied startup.py at ~/.casa/startup.py

Using matplotlib backend: agg
Creating a new telemetry file
casaVersion =  6.5.6.22
imported casatasks and casatools individually
CASA 6.5.6.22 -- Common Astronomy Software Applications [6.5.6.22]
CASA version:  6.5.6
[INFO] [        build_key_handler] 
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] Initializing the PHANGS-ALMA pipeline KeyHandler.
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] 
[INFO] [         _read_master_key] Reading the master key.
[INFO] [         _read_master_key] Master key file: /arc/projects/salvage/phangs_imaging

## Run PHANGS moments

In [ ]:
# select appropriate resources

image = "images.canfar.net/skaha/astroml:latest"
#image = "images.canfar.net/casa-5/casa:5.6.1-8-pipeline"
cmd = '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh'
ram=4
cores=2

session_ids = []

# testing targets that have empty mom0 maps
#rerun_targets = ['587726033309335775', '588010359075700997', '587727944034091239']
rerun_targets = ['587727944034091201','587727944034091092']

# loop over galaxies and launch jobs
for i in range(len(objID_sample)):

    ID = objID_sample[i]
    NAME = name_sample[i]

    # running only on problem galaxies
    #if ID not in rerun_targets:
    #    continue

    # launch headless session on the science platform
    session = Session()
    session_id = session.create(
        name  = ID,
        image = image,
        cores = cores,
        ram   = ram,
        kind  = "headless",
        cmd   = cmd,
        args  = f'{NAME} {ID}'
    )

    print("Sesion ID: {}".format(session_id[0]))
    session_ids.append(session_id[0])


2024-03-16 00:00:41,343 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:00:41,343 INFO Creating 1 session(s) with parameters:
2024-03-16 00:00:41,345 - skaha-client-skaha.session - INFO - {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '587731173842027508_8623-6104 587731173842027508', 'env': {}}
2024-03-16 00:00:41,345 INFO {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '587731173842027508_8623-6104 587731173842027508', 'env': {}}


Sesion ID: udb7xscs


2024-03-16 00:00:45,248 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:00:45,248 INFO Creating 1 session(s) with parameters:
2024-03-16 00:00:45,250 - skaha-client-skaha.session - INFO - {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8623-6104 587731173842027508', 'env': {}}
2024-03-16 00:00:45,250 INFO {'name': '587731173842027508', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8623-6104 587731173842027508', 'env': {}}


Sesion ID: krwcyrob


2024-03-16 00:00:47,596 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:00:47,596 INFO Creating 1 session(s) with parameters:
2024-03-16 00:00:47,597 - skaha-client-skaha.session - INFO - {'name': '587731185665638418', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8655-3701 587731185665638418', 'env': {}}
2024-03-16 00:00:47,597 INFO {'name': '587731185665638418', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8655-3701 587731185665638418', 'env': {}}


...

2024-03-16 00:01:53,682 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:01:53,682 INFO Creating 1 session(s) with parameters:
2024-03-16 00:01:53,685 - skaha-client-skaha.session - INFO - {'name': '587727221402042402', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '587727221402042402_7977-3703 587727221402042402', 'env': {}}
2024-03-16 00:01:53,685 INFO {'name': '587727221402042402', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '587727221402042402_7977-3703 587727221402042402', 'env': {}}


Sesion ID: nht5srg7
Sesion ID: znwvzl9t


2024-03-16 00:01:56,223 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:01:56,223 INFO Creating 1 session(s) with parameters:
2024-03-16 00:01:56,225 - skaha-client-skaha.session - INFO - {'name': '587727221396013696', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8618-9102 587727221396013696', 'env': {}}
2024-03-16 00:01:56,225 INFO {'name': '587727221396013696', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': '8618-9102 587727221396013696', 'env': {}}


Sesion ID: lago5657


2024-03-16 00:02:04,072 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:02:04,072 INFO Creating 1 session(s) with parameters:
2024-03-16 00:02:04,073 - skaha-client-skaha.session - INFO - {'name': '587727944034091201', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'COSMOS-1197519 587727944034091201', 'env': {}}
2024-03-16 00:02:04,073 INFO {'name': '587727944034091201', 'image': 'images.canfar.net/skaha/astroml:latest', 'cores': 2, 'ram': 4, 'kind': 'headless', 'cmd': '/arc/projects/salvage/ALMA_reduction/bash_scripts/run_PHANGS_moments.sh', 'args': 'COSMOS-1197519 587727944034091201', 'env': {}}
2024-03-16 00:02:06,623 - skaha-client-skaha.session - INFO - Creating 1 session(s) with parameters:
2024-03-16 00:02:06,623 INFO Creating 1 session(s) with parameters:
2024-03-16 00:02:06,624 - skaha-client-skaha.se

In [88]:
for i,sid in enumerate(session_ids):

    info = session.info(sid)[0]
    print(info['id'], info['status'])

kbh32t5q Running
f7yvvody Running
ykxoanw6 Running
n39zo5xd Running
ej5btulv Running


In [152]:
for sid in session_ids:
    
    print(session.logs(sid)[sid])

Execute bash script.
NAME = 587731173842027508_8623-6104
OBJID = 587731173842027508
Downgrading astropy
Defaulting to user installation because normal site-packages is not writeable
  Using cached astropy-5.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl.metadata (9.5 kB)
Using cached astropy-5.3-cp310-cp310-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (10.1 MB)
ERROR: Could not install packages due to an OSError: [Errno 2] No such file or directory: '/arc/home/swilkinson/.local/lib/python3.10/site-packages/astropy/io/votable/data/ucd1p-words.txt'


[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: pip install --upgrade pip
Running derived pipeline...
[DEBU] [                 <module>] casa_enabled = False
[INFO] [        build_key_handler] 
[INFO] [        build_key_handler] &%&%&%&%&%&%&%&%&%&%&%&%&%&%&&%&%&%&%&%&%&%&%&%&%
[INFO] [        build_key_handler] I